In [1]:
%matplotlib widget
import pytest
import ipytest
ipytest.autoconfig()

from preliz import Roulette

In [2]:
@pytest.mark.parametrize("x_min, x_max, nrows, ncols, figsize, dist_names, params", [
    (0, 10, 10, 10, None, None, None),  # Test default behavior
    (-5, 5, 10, 10, None, None, None),  # Test different domain
    (0, 10, 5, 5, None, None, None),  # Test different grid dimensions
    (0, 10, 10, 10, (10, 8)),  # Test custom figsize
    (0, 10, 10, 10, None, ["Normal", "StudentT"], "Normal(mu=0), StudentT(nu=0.001)"),  # Test custom dist and params
])
def test_roulette(x_min, x_max, nrows, ncols, figsize):
    Roulette(x_min, x_max, nrows, ncols, figsize)

In [ ]:
def test_roulette_initialization():
    roulette = Roulette(x_min=0, x_max=10, nrows=10, ncols=11)
    assert roulette._x_min == 0
    assert roulette._x_max == 10
    assert roulette._nrows == 10
    assert roulette._ncols == 11
    assert roulette._figsize == (8, 6)


def test_roulette_update_grid():
    roulette = Roulette(x_min=0, x_max=10, nrows=10, ncols=11)
    roulette._widgets['w_x_min'].value = 1
    roulette._widgets['w_x_max'].value = 9
    roulette._widgets['w_nrows'].value = 8
    roulette._widgets['w_ncols'].value = 9
    roulette._update_grid()
    assert roulette._x_min == 1
    assert roulette._x_max == 9
    assert roulette._nrows == 8
    assert roulette._ncols == 9


def test_roulette_weights_to_ecdf():
    roulette = Roulette(x_min=0, x_max=10, nrows=10, ncols=11)
    roulette._grid._weights = {0: 2, 1: 6, 2: 10, 3: 10, 4: 7, 5: 3, 6: 1, 7: 1, 8: 1, 9: 1}
    x_vals, cum_sum, probabilities, mean, std, filled_columns = roulette._weights_to_ecdf()
    assert len(x_vals) == 10
    assert len(cum_sum) == 10
    assert len(probabilities) == 10
    assert filled_columns == 10


def test_roulette_on_leave_fig():
    roulette = Roulette(x_min=0, x_max=10, nrows=10, ncols=11)
    roulette._grid._weights = {0: 2, 1: 6, 2: 10, 3: 10, 4: 7, 5: 3, 6: 1, 7: 1, 8: 1, 9: 1}
    roulette._widgets['w_distributions'].value = ["Gamma", "LogNormal", "StudentT", "BetaScaled", "Normal"]
    roulette._widgets['w_repr'].value = "pdf"
    roulette._on_leave_fig()
    assert roulette.dist is not None
    assert roulette.hist is not None